### This file contains a preview of all the data pulled for S&P500 companies

![ratios definition](Ratios_def.png)

In [14]:
import datetime
import pandas as pd
import wrds

# Connect to WRDS
conn = wrds.Connection()

# Date
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2020, 12, 31)

#print(conn.list_libraries())

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
Loading library list...
Done


In [3]:
sp500ConstituentsFile = 'sp500_constituents.csv'
sp500Constituents = pd.read_csv(sp500ConstituentsFile)

In [4]:
# get a list of tickers
tickers = sp500Constituents['Symbol'].tolist()

tickerString = []
for ticker in tickers:
    tickerString.append("ticker = '" + ticker + "'")

tickerString = ' or '.join(tickerString)

# get the data
firmRatioDf = conn.raw_sql("select * from wrdsapps_finratio.firm_ratio where " + tickerString )

In [40]:
# Save to csv
saveToFile = False
if(saveToFile):
    firmRatioDf.to_csv('firm_ratio.csv')
else:
    firmRatioDf = pd.read_csv('firm_ratio.csv')


In [41]:
# treat public_date as date
firmRatioDf['public_date'] = pd.to_datetime(firmRatioDf['public_date'])

firmRatioDf['public_date'].value_counts()


2021-12-31    435
2021-04-30    435
2021-02-28    435
2021-03-31    435
2021-09-30    434
             ... 
1970-02-28    145
1970-09-30    145
1970-04-30    145
1970-08-31    145
1970-01-31    143
Name: public_date, Length: 624, dtype: int64

In [42]:
# treat public_date as date
firmRatioDf['public_date'] = pd.to_datetime(firmRatioDf['public_date'])

latestOnly = firmRatioDf[firmRatioDf['public_date'] == datetime.datetime(2017, 2, 28)]

columnsOfInterest = ['public_date', 'ticker', 'roa', 'intcov_ratio', 'quick_ratio', 'intcov', 'bm', 'npm', 'debt_assets', 'debt_ebitda', 'inv_turn', 'cash_ratio']

ratiosOfInterestDf = latestOnly[columnsOfInterest]

ratiosOfInterestDf

,public_date,ticker,roa,intcov_ratio,quick_ratio,intcov,bm,npm,debt_assets,debt_ebitda,inv_turn,cash_ratio
653,2017-02-28,ABT,0.105949,17.004695,3.654955,5.990610,0.369329,0.050976,0.606634,4.423317,3.222730,2.819069
1380,2017-02-28,PNW,0.086478,4.160918,0.417366,3.148717,0.916132,0.126343,0.691587,3.222494,7.668622,0.006869
1475,2017-02-28,AAL,0.157149,5.798263,0.665369,3.583012,0.160266,0.066600,0.926181,3.107877,26.259581,0.504397
1862,2017-02-28,AMD,-0.018351,-0.833333,1.321694,-2.185897,0.040272,-0.116339,0.874737,-24.322034,4.006998,0.939079
2678,2017-02-28,APD,0.140038,10.662610,0.849159,8.235970,0.250545,0.142316,0.577241,2.274587,6.610056,0.217685
...,...,...,...,...,...,...,...,...,...,...,...,...
175994,2017-02-28,HII,0.165320,11.013158,1.438570,8.539474,0.194252,0.081070,0.739767,1.249267,21.907071,0.536113
176154,2017-02-28,ENPH,-0.293433,-21.248828,1.037839,-23.328164,0.020670,-0.209125,0.992053,-0.702081,6.980346,0.214614
176273,2017-02-28,FBHS,0.155610,13.721757,1.084599,9.627615,0.307874,0.082730,0.539241,1.838043,5.663753,0.298410
176389,2017-02-28,EPAM,0.186575,NaN,5.563213,NaN,0.238699,0.085564,0.155970,0.157541,NaN,3.139142


In [11]:
# save to csv
ratiosOfInterestDf.to_csv('ratios_of_interest_sp500.csv')

In [18]:
# Open credit ratings file
creditRatingsFile = "credit_ratings_raw.csv"

creditRatingsDf = pd.read_csv(creditRatingsFile)

In [20]:
creditRatingsDf = creditRatingsDf[creditRatingsDf['tic'].isin(tickers)]

In [35]:
creditRatingsDf['datadate'] = pd.to_datetime(creditRatingsDf['datadate'])

print(creditRatingsDf['datadate'].max())

# get the latest credit rating
latestCreditRatingsDf = creditRatingsDf[creditRatingsDf['datadate'] == creditRatingsDf['datadate'].max()]

# rename splticrm to long term credit rating
latestCreditRatingsDf = latestCreditRatingsDf.rename(columns={'splticrm': 'long_term_credit_rating'})

latestCreditRatingsDf = latestCreditRatingsDf[['tic', 'long_term_credit_rating']]

2017-02-28 00:00:00


In [44]:
# Merge credit ratings with ratios
mergedDf = pd.merge(ratiosOfInterestDf, latestCreditRatingsDf, left_on='ticker', right_on='tic', how='inner')

mergedDf.drop(columns=['tic'], inplace=True)

mergedDf

,public_date,ticker,roa,intcov_ratio,quick_ratio,intcov,bm,npm,debt_assets,debt_ebitda,inv_turn,cash_ratio,long_term_credit_rating
0,2017-02-28,ABT,0.105949,17.004695,3.654955,5.990610,0.369329,0.050976,0.606634,4.423317,3.222730,2.819069,BBB
1,2017-02-28,PNW,0.086478,4.160918,0.417366,3.148717,0.916132,0.126343,0.691587,3.222494,7.668622,0.006869,A-
2,2017-02-28,AAL,0.157149,5.798263,0.665369,3.583012,0.160266,0.066600,0.926181,3.107877,26.259581,0.504397,BB-
3,2017-02-28,AMD,-0.018351,-0.833333,1.321694,-2.185897,0.040272,-0.116339,0.874737,-24.322034,4.006998,0.939079,CCC+
4,2017-02-28,APD,0.140038,10.662610,0.849159,8.235970,0.250545,0.142316,0.577241,2.274587,6.610056,0.217685,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,2017-02-28,TSLA,0.019603,-2.629887,0.719468,-1.749129,0.137924,-0.096414,0.739062,24.069734,2.662713,0.600435,B-
334,2017-02-28,MPC,0.102459,3.966454,0.664008,2.875399,0.655186,0.021027,0.522595,2.357716,9.143277,0.124825,BBB
335,2017-02-28,HII,0.165320,11.013158,1.438570,8.539474,0.194252,0.081070,0.739767,1.249267,21.907071,0.536113,BB+
336,2017-02-28,FBHS,0.155610,13.721757,1.084599,9.627615,0.307874,0.082730,0.539241,1.838043,5.663753,0.298410,BBB+


In [45]:
# save to csv
mergedDf.to_csv('ratios_with_credit_ratings_sp500.csv')